In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install streamlit -q
!pip install pyngrok
!pip install pyxlsb

In [ ]:
## 필요 라이브러리 설치
!pip install transformers
!pip install sentence_transformers

In [4]:
cd "/content/drive/MyDrive/final project/code/streamlit/"

/content/drive/MyDrive/final project/code/streamlit


In [17]:
%%writefile BERT.py

import streamlit as st

import urllib.request
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

## 경고 해제
pd.set_option('mode.chained_assignment',  None)

train_data = pd.read_excel('/content/drive/MyDrive/final project/data/Sentence(100)_Embedding.xlsb')
t_data = train_data[['Q','A','embedding']]

## t_data['embedding'] str -> numpy 형변환
a = []
for _ in range(len(t_data)):
  tmp = t_data['embedding'][_].replace('[', '').replace(']', '').replace('\n', '')
  s_to_n = np.fromstring(tmp, dtype='f', sep=' ')
  a.append(np.array(s_to_n, dtype='f'))
  if _ % 17574 == 0 :
    print("\r", end='')
    print(f'{_ // int(len(t_data) / 10) * 10}% :', '##'*(_ // int(len(t_data) / 10)), end='')

new_data = t_data[['Q','A']]
new_data['embedding'] = a

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_similar_answer(input):
    embedding = model.encode(input)
    new_data['score'] = new_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return new_data.loc[new_data['score'].idxmax()]['A']

## Text Input
st.text("안녕하세요 00님! 오늘의 일기를 작성해주세요")
message = st.text_area("일기 작성 칸")
st.text("기록이 완료됐습니다.")

## 위로 문장 출력
st.write(return_similar_answer(message))

Overwriting BERT.py


In [18]:
from pyngrok import ngrok
ngrok.set_auth_token('2KqC6FInSv5YFHXU6mnORM4NadD_76hbXWt9hmxC9HjWksch')

In [19]:
!nohup streamlit run BERT.py --server.port 80 &
url = ngrok.connect(port ='80')
url

nohup: appending output to 'nohup.out'


<NgrokTunnel: "http://4743-34-105-36-8.ngrok.io" -> "http://localhost:80">

In [21]:
ngrok.kill() # url 초기화

In [9]:
!streamlit run BERT.py & npx localtunnel --port 8501

[..................] - rollbackFailedOptional: verb npm-session 7f1c575d904dacb


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.105.36.8:8501

npx: installed 22 in 3.367s
your url is: https://happy-dancers-rescue-34-105-36-8.loca.lt
2023-02-02 05:38:02.098 Load pretrained SentenceTransformer: sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens
2023-02-02 05:38:42.870 Use pytorch device: cuda
2023-02-02 05:38:43.508 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/range.py", line 385, in get_loc
    return self._range.index(new_key)
ValueError: 100 is not in range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 565, in _run_script
    exec(code, module.__dict__)
  F